In [1]:
!CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=53321393 sha256=9c0a5c046be3f09baa0c1eb8f13603456b8989cdafafaa8bcdfe76fe4adedaed
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [2]:
!huggingface-cli download Qwen/Qwen2-7B-Instruct-GGUF qwen2-7b-instruct-q5_k_m.gguf --local-dir /content/ --local-dir-use-symlinks False

/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
qwen2-7b-instruct-q5_k_m.gguf: 100% 5.44G/5.44G [00:31<00:00, 173MB/s]
Download complete. Moving file to /content/qwen2-7b-instruct-q5_k_m.gguf
/content/qwen2-7b-instruct-q5_k_m.gguf


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
# MODELS.PY
import torch
from llama_cpp import Llama


class GGUFModel:
    """
    Class to handle the GGUF model.
    """

    def __init__(self, gguf_model_path: str, system_prompt: str, context_window_size: int,
                 verbose: bool = False, n_batch: int = 4) -> None:
        """
        Initializes the model and its relevant parameters.
        """
        try:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            self.system_prompt = system_prompt
            self.model = Llama(
                model_path=gguf_model_path,
                n_gpu_layers= -1 if self.device=='cuda' else 0,
                n_batch=n_batch,
                n_ctx=context_window_size,
                verbose=verbose
            )
            if self.device == 'cuda':
                print(f"Model located at {gguf_model_path} loaded successfully on GPU.")
            else:
                print(f"Model located at {gguf_model_path} loaded successfully on CPU.")
        except Exception as e:
            raise RuntimeError(f"An unexpected error occured while trying to load the GGUF model: {str(e)}")

    def perform_inference(self, instruction_prompt: str) -> str:
        """
        Performs inference on the given instruction prompt and returns the model output.
        """
        try:
            messages = [
                {"role": "user", "content": f"{instruction_prompt}"}
            ]
            if self.system_prompt is not None:
                messages.insert(0, {"role": "system", "content": self.system_prompt})
            output = self.model.create_chat_completion(
                messages=messages,
            )
            text = output['choices'][0]['message']['content']
            return text
        except Exception as e:
            raise RuntimeError(f"An unexpected error occured while trying to perform inference: {str(e)}")


In [2]:
# PROMPTS.PY
############################
# DATASET LABEL GENERATION #
############################
def get_label_generation_system_prompt() -> str:
    """
    Gives system prompt for dataset label (resume match score) generation.
    """
    return """
    You are an AI-powered Resume Evaluator designed to assess how well a candidate's resume matches a given job description. You follow structured evaluation criteria, provide fair assessments, and return responses in a machine-readable format. Your outputs are clear, concise, and logically structured.

    Always ensure that:
    - Your evaluations are objective, unbiased, and based strictly on the provided inputs.
    - Your responses follow a structured JSON format for easy parsing.
    - You consider relevant skills, experience, education, and responsibilities while scoring the match.
    - Minor differences in wording are accounted for, ensuring fair scoring.

    Your primary role is to generate structured resume-job fit assessments that help automate candidate evaluation efficiently.
    """

def get_label_generation_instruction_prompt(resume: str, jd: str) -> str:
    """
    Gives instruction prompt for dataset label (resume match score) generation.
    """
    return f"""
    You are an AI-powered Resume Evaluator, tasked with analyzing how well a given resume matches a specific job description. Your response must be structured and contain a detailed breakdown of the match.

    **Instructions:**
    - Evaluate the resume against the job description based on multiple factors such as skills, experience, qualifications, and key responsibilities.
    - Only return the JSON as output and nothing extra.
    - Provide a structured JSON output with the following fields:

    **Output Format:**
    {{
        "summary": "A brief summary of how well the resume matches the job description.",
        "match_score": "A percentage (0-100) indicating the overall match strength.",
        "skill_match": {{
            "matched": ["List of skills from the JD found in the resume"],
            "missing": ["List of important skills from the JD missing in the resume"],
            "score": "A percentage score (0-100) based on skill relevance."
        }},
        "experience_match": {{
            "matched_years": "Number of years of relevant experience found in the resume.",
            "required_years": "Number of years required as per the JD.",
            "score": "A percentage score (0-100) indicating experience match."
        }},
        "education_match": {{
            "matched_degree": "Degree(s) from the resume that match the JD requirements.",
            "required_degree": "Degree(s) specified in the JD.",
            "score": "A percentage score (0-100) for education match."
        }},
        "responsibility_match": {{
            "matched": ["Key responsibilities from the JD found in the resume"],
            "missing": ["Key responsibilities missing from the resume"],
            "score": "A percentage score (0-100) indicating responsibility match."
        }},
        "final_assessment": "A brief verdict on whether the candidate is a strong, moderate, or weak fit."
    }}

    **Evaluation Guidelines:**
    - Consider exact and semantic similarity while matching skills, experience, and responsibilities.
    - Give higher scores for a strong match but ensure fairness in assessment.
    - Do not be overly strict; minor variations in terminology should still be considered a match.
    - Ensure logical scoring where no single category heavily skews the overall score.

    **Now, evaluate the following resume against the job description:**
    **Resume:**
    {resume}

    **Job Description:**
    {jd}

    Provide your structured JSON response accordingly.
    """


In [ ]:
# PREDICT_SCORES.PY
import os
import time
import json
import pandas as pd
# from models import GGUFModel
# from prompts import get_label_generation_system_prompt, get_label_generation_instruction_prompt


class DatasetCompleterAutomatic:
    """
    Uses a base model to get and save resume-jd matching scores (label)
    on the incomplete dataset.
    """

    def __init__(self, dataset_path: str, output_path: str, gguf_model_path: str, system_prompt: str, context_window_size) -> None:
        """
        Initialises the parameters needed for dataset completion.
        """
        self.model_handler = GGUFModel(
            gguf_model_path=gguf_model_path,
            system_prompt=system_prompt,
            context_window_size=context_window_size
        )
        self.dataset = pd.read_excel(dataset_path)

        self.output_store_path = output_path

        if os.path.exists(self.output_store_path):
            self.output_dict = pd.read_excel(self.output_store_path).to_dict(orient="list")
            self.starting_index = len(self.output_dict['JD'])
            print(f"Resuming from row {self.starting_index + 1}...")
        else:
            self.output_dict = {
                'JD': [],
                'Resume': [],
                'Response': []
            }
            self.starting_index = 0

    def save_current_output_dict(self) -> None:
        """
        Saves current version of the output as an excel file in the provided directory.
        """
        output_df = pd.DataFrame(self.output_dict)
        output_df.to_excel(self.output_store_path, index=False)

    def __call__(self) -> None:
        """
        Uses the specified model to predict and validate the output and
        store it in the excel file.
        """
        for index, row in self.dataset.iloc[self.starting_index:].iterrows():
            try:
                print(f"\n\nProcessing row {index + 1} out of {len(self.dataset)}...\n\n")
                resume = row['Resume']
                jd = row['JD']
                instruction_prompt = get_label_generation_instruction_prompt(resume=resume, jd=jd)
                inference_start_time = time.time()
                response = self.model_handler.perform_inference(instruction_prompt=instruction_prompt)
                print(f"Inference time taken: {(time.time() - inference_start_time):.2f} seconds")
                print(response)
                try:
                    parsed_response = json.loads(response)

                    required_keys = ["match_score", "summary", "skill_match", "experience_match"]
                    if not all(key in parsed_response for key in required_keys):
                        raise ValueError(f"Response JSON missing required fields! Response: {parsed_response}")
                except json.JSONDecodeError:
                    raise Exception(f"Invalid JSON response!")

                self.output_dict['JD'].append(jd)
                self.output_dict['Resume'].append(resume)
                self.output_dict['Response'].append(response)
                self.save_current_output_dict()
            except Exception as e:
                print(f"Skipping row {index+1}: {str(e)}")
                continue


if __name__ == '__main__':
    dataset_completer = DatasetCompleterAutomatic(
        dataset_path="/content/dataset_without_labels.xlsx",
        output_path="/content/drive/MyDrive/dataset.xlsx",
        gguf_model_path="/content/qwen2-7b-instruct-q5_k_m.gguf",
        system_prompt=get_label_generation_system_prompt(),
        context_window_size=8000
    )
    dataset_completer()


llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_init_from_model: n_ctx_per_seq (8000) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


Model located at /content/qwen2-7b-instruct-q5_k_m.gguf loaded successfully on GPU.
Resuming from row 4...


Processing row 5 out of 20760...


Inference time taken: 39.58 seconds
{
    "summary": "The resume shows a background in computer science with relevant skills in data science, programming, and database management. However, it lacks the required master's degree and extensive experience in advanced problem-solving and critical thinking, which are key responsibilities for the role.",
    "match_score": "60",
    "skill_match": {
        "matched": ["Data Science", "Data Analysis", "Python", "Core Java", "Database Management"],
        "missing": ["Advanced problem-solving", "Critical thinking"],
        "score": "80"
    },
    "experience_match": {
        "matched_years": "Less than 1 year",
        "required_years": "Not specified",
        "score": "100"
    },
    "education_match": {
        "matched_degree": ["MCA"],
        "required_degree": ["Master's degree in Computer 